# Retrieval Evaluation

In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
# os.environ["TMPDIR"] = "/srv/data/tmp"
# os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install "haystack-ai>=2.15.0rc1"
%pip install ragas-haystack
%pip install nltk
%pip install openai
%pip install pandas
%pip install ragas-haystack
%pip install "sentence-transformers>=3.0.0"
%pip install hf_xet
%pip install "ollama-haystack==2.4.2"
%pip install tqdm # For Progress Bar
%pip install einops

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from haystack.document_stores.in_memory import InMemoryDocumentStore
import importlib
from pipelines.components.evaluators import DocumentMRREvaluator, DocumentMAPEvaluator, DocumentRecallEvaluator
from tqdm import tqdm
tqdm.pandas()
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S'
)

In [3]:
# from config.secret import OPENAI_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

os.environ["EMBEDDING_MODEL_NAME"] = "Qwen/Qwen3-Embedding-4B"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = "./model-assets/sentence-transformers"
os.environ["HF_HUB_CACHE"] = "./model-assets/hugging-face"
os.environ["EMBEDDING_MODEL_TOP_K"] = "10"

os.environ["LLM_NAME"] = "gemma3:12b"
os.environ["LLM_CONTEXT_SIZE"] = "8192"

In [4]:
df = pd.read_pickle("data/qa_with_docs_flat/question_answers_docs_line_1_0_dataset_flat.pkl")

## Setup Retrieval Pipelines

Sets up the basic / hybrid RAG pipelines. These can then be used later in combintation with the different indexing methods from before to evaluate different RAG setups.

In [5]:
base_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/docs_line_1_0_base_indexing_store.json")
context_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/docs_line_1_0_context_indexing_store.json")

In [6]:
context_indexing_store.filter_documents()

[Document(id=5185ded672f9760cc64f96058f6b9a7df27f7aff2d7bef6ddfae2dd661c5dc72, content: 'Die "Schädellage" ist die ideale Position für eine vaginale Geburt, da der Kopf des Babys zuerst geb...', meta: {'title': 'Äußere Wendung.md', 'source_id': 'b18da279-2022-47e8-a0be-835a32d2a215', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}, embedding: vector of size 2560),
 Document(id=cf66cea8dc01d863e343bdba095739a0daa1943889d83f8ce9499b4ca79237e8, content: 'Eine Beckenendlage bedeutet, dass das Baby mit dem Becken (statt mit dem Kopf) nach unten im Mutterl...', meta: {'title': 'Äußere Wendung.md', 'source_id': 'b18da279-2022-47e8-a0be-835a32d2a215', 'page_number': 1, 'split_id': 2, 'split_idx_start': 452}, embedding: vector of size 2560),
 Document(id=b73dc03408f36fb40bc97647be966f5f2b431e68ab07e2010131df3e78e0be54, content: 'Diese alternativen Methoden sind oft nicht wissenschaftlich belegt und werden von Hebammen durchgefü...', meta: {'title': 'Äußere Wendung.md', 'source_id': 'b

In [7]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)
from pipelines.components.wrap_in_list_adapter import WrapInListAdapter

base_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(base_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

base_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
base_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
base_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
base_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
base_retrieval_pipeline.connect("reranker", "retriever_to_evaluator_adapter")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

12:46:38 INFO PyTorch version 2.7.0 available.
12:46:38 INFO Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-4B


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

12:46:44 INFO 2 prompts are loaded, with the keys: ['query', 'document']


TypeError: QwenYesNoReranker.__init__() got an unexpected keyword argument 'model_kwargs'

In [ ]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(base_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

hybrid_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
hybrid_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
hybrid_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
hybrid_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
hybrid_retrieval_pipeline.connect("reranker", "retriever_to_evaluator_adapter")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-Reranker-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - reranker: SentenceTransformersSimilarityRanker
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - joiner.documents -> reranker.documents (List[Document])
  - reranker.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_ada

In [ ]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)

context_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(context_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

context_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
context_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
context_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
context_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
context_retrieval_pipeline.connect("reranker", "retriever_to_evaluator_adapter")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-Reranker-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reranker: SentenceTransformersSimilarityRanker
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reranker.documents (List[Document])
  - reranker.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> recall_evaluator.retrieved_documents (List[List[Document]])

In [ ]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

context_hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(context_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

context_hybrid_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
context_hybrid_retrieval_pipeline.connect("reranker", "retriever_to_evaluator_adapter")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-Reranker-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - reranker: SentenceTransformersSimilarityRanker
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - joiner.documents -> reranker.documents (List[Document])
  - reranker.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_ada

## Run Evaluation

In [ ]:
test_cases = [
    {
        "name": "Basic RAG",
        "pipeline": base_retrieval_pipeline,
    },
    {
        "name": "Hybrid RAG",
        "pipeline": hybrid_retrieval_pipeline,
    },
    {
        "name": "Contextual RAG",
        "pipeline": context_retrieval_pipeline,
    },
    {
        "name": "Contextual Hybrid RAG",
        "pipeline": context_hybrid_retrieval_pipeline,
    },
]

In [ ]:
import logging

# silence haystack’s pipeline logs
logging.getLogger("haystack").setLevel(logging.WARNING)
logging.getLogger("haystack.core.pipeline").setLevel(logging.WARNING)

# if you see similar spam from transformers, ragas, etc.
logging.getLogger("transformers").setLevel(logging.WARNING)
logging.getLogger("ragas").setLevel(logging.WARNING)

In [ ]:
import config.prompt
importlib.reload(config.prompt)

top_k_values = [5, 10, 20]

results = []

# 1) Filter out the null‐question rows
df_nonnull = df[df["question"].notnull()]

df_shuffled = df_nonnull.sample(n=50, random_state=42).reset_index(drop=True)

for top_k in top_k_values:
    for index, row in tqdm(
        df_shuffled.iterrows(),
        total=len(df_shuffled),
        desc="Processing rows",
        unit="row"
    ):
        reference = row["groundTruth"]
        relevant_documents = row["documents"]
        question = row["question"]

        for test_case in test_cases:
            pipeline = test_case["pipeline"]
            request_payload = {
                "retriever": {
                    "top_k": top_k,
                },
                "reranker": {
                    "query": question,
                    "top_k": top_k,
                },
                "query_embedder": {
                    "text": question,
                },
                "map_evaluator": {
                    "ground_truth_documents": [relevant_documents],
                },
                "mrr_evaluator": {
                    "ground_truth_documents": [relevant_documents],
                },
                "recall_evaluator": {
                    "ground_truth_documents": [relevant_documents],
                }
            }
            if "bm25_retriever" in pipeline.graph.nodes:
                request_payload["bm25_retriever"] = {
                    "query": question,
                    "top_k": top_k,
                }
            result = pipeline.run(request_payload)

            map_score = result.get("map_evaluator", {}).get("score", {})
            mrr_score = result.get("mrr_evaluator", {}).get("score", {})
            recall_score = result.get("recall_evaluator", {}).get("score", {})

            df.at[index, f"{test_case['name']}_map"] = map_score
            df.at[index, f"{test_case['name']}_mrr"] = mrr_score
            df.at[index, f"{test_case['name']}_recall"] = recall_score

    embedding_model_name = os.environ['EMBEDDING_MODEL_NAME'].replace('/', '')
    df.to_pickle(f"results/retrieval_eval_results_{embedding_model_name}_topk{top_k}.pkl")

Processing rows:   0%|          | 0/50 [00:00<?, ?row/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 50/50 [02:39<00:00,  3.19s/row]


In [ ]:
df

,question,references,groundTruth,source_file,documents,variant,prev_messages,Basic RAG_map,Basic RAG_mrr,Basic RAG_recall,Hybrid RAG_map,Hybrid RAG_mrr,Hybrid RAG_recall,Contextual RAG_map,Contextual RAG_mrr,Contextual RAG_recall,Contextual Hybrid RAG_map,Contextual Hybrid RAG_mrr,Contextual Hybrid RAG_recall
0,Wie läuft eine äußere Wendung ab und was passi...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=42e5316808b9457eed1b32ec3d3a591b2...,default,[],0.052632,0.052632,1.0,0.000000,0.000000,0.0,1.000000,1.000000,1.0,0.000000,0.000000,0.0
1,Und wie lange dauert dieser Eingriff normalerw...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=42e5316808b9457eed1b32ec3d3a591b2...,contextual,"[Wie läuft eine äußere Wendung ab?, Bei der äu...",0.216667,0.333333,1.0,0.225490,0.333333,1.0,0.666667,1.000000,1.0,0.000000,0.000000,0.0
2,"Ey, wie machen die eigentlich so 'ne äußere We...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=42e5316808b9457eed1b32ec3d3a591b2...,slang,[],0.166667,0.166667,1.0,0.083333,0.083333,1.0,0.071429,0.071429,1.0,0.052632,0.052632,1.0
3,How is an external version done and what happe...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=42e5316808b9457eed1b32ec3d3a591b2...,english,[],0.111111,0.111111,1.0,0.250000,0.250000,1.0,0.125000,0.125000,1.0,0.000000,0.000000,0.0
4,"Können Sie mir erklären, wie eine äußere Wendu...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=42e5316808b9457eed1b32ec3d3a591b2...,german-variation-1,[],0.113095,0.083333,1.0,0.166667,0.166667,1.0,1.000000,1.000000,1.0,0.071429,0.071429,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,"Was ist zu tun, wenn nach der Geburtseinleitun...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8cbfd372f826aeb040aa026c9155ccd35...,german-variation-2,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,"Kann mir jemand sagen, was ich machen soll, we...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8cbfd372f826aeb040aa026c9155ccd35...,german-variation-3,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,Ich hatte eine Geburtseinleitung – was mache i...,[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8cbfd372f826aeb040aa026c9155ccd35...,german-variation-4,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,"Was empfehlen Sie, wenn nach der Einleitung de...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8cbfd372f826aeb040aa026c9155ccd35...,german-variation-5,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
